# Moosic Baseline Model :: Iteration v1


* combining datasets as done prior [data preprocessing][data management]
* train test split [modelling]
* baseline model [discussion needed]
* baseline model sketch and implementation


## Importing required libraries




In [109]:
# IMPORT LIBRARIES


try:

    import numpy as np
    import pandas as pd

    # databases - sql
    #from dotenv import dotenv_values
    #import sqlalchemy

    # visualisation
    import seaborn as sns
    import matplotlib.pyplot as plt

    # modelling - evaluation metrics
    from sklearn.model_selection import train_test_split
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler

    from sklearn.metrics import euclidean_distances
    from sklearn.metrics.pairwise import sigmoid_kernel
    from sklearn.metrics.pairwise import cosine_similarity


    from sklearn.pipeline import Pipeline
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA


except ImportError as error:
    print(f"Installation of the required dependencies necessary! {error}")

    %pip install numpy
    %pip install pandas
    #%pip install dotenv
    #%pip install sqlalchemy
    %pip install seaborn
    %pip install matplotlib
    %pip install sklearn

    print(f"Successful installation of the required dependencies necessary")


    import warnings
    warnings.filterwarnings('ignore')



## Loading the data

In [110]:
df_artists = pd.read_csv('../data/moosic-raw/spotify_600k_artists.csv', low_memory=False)
df_artists.head().T

,0,1,2,3,4
id,0DheY5irMjBUeLybbCUEZ2,0DlhY15l3wsrnlfGio2bjU,0DmRESX2JknGPQyO15yxg7,0DmhnbHjm1qw6NCYPeZNgJ,0Dn11fWM7vHQ3rinvWEl4E
followers,0.0,5.0,0.0,0.0,2.0
genres,[],[],[],[],[]
name,Armid & Amir Zare Pashai feat. Sara Rouzbehani,ปูนา ภาวิณี,Sadaa,Tra'gruda,Ioannis Panoutsopoulos
popularity,0,0,0,0,0


In [111]:
df_tracks = pd.read_csv('../data/moosic-raw/spotify_600k_tracks.csv', low_memory=False)
df_tracks.head().T

,0,1,2,3,4
id,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
name,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
popularity,6,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
explicit,0,0,0,0,0
artists,['Uli'],['Fernando Pessoa'],['Ignacio Corsini'],['Ignacio Corsini'],['Dick Haymes']
id_artists,['45tIt06XoI0Iio4LBEVpls'],['14jtPCOoNZwquk5wd9DxrY'],['5LiOoJbxVSAMkBS2fUm3X2'],['5LiOoJbxVSAMkBS2fUm3X2'],['3BiJGZsyX9sJchTqcSA7Su']
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


## Data Overview Artists

| column | additional information |
|--------|------------------------|
| id | id of artist |
| followers | number of followers | 
| genres | genres associated with artist |
| name | name of artist |
| popularity | popularity of artist in range 0 to 100 |

## Data Overview Tracks

| column | additional information |
|--------|------------------------|
| id | id of track |
| name | name of track | 
| popularity | popularity of track in range 0 to 100 |
| duration_ms | duration of songs in ms |
| explicit | whether it contains explicit content or not |
| artists | artists who created the track | 
| id_artists | id of artists who created the track |
| release_date | date of release |
| danceability | how danceable a song is in range 0 to 1 |
| energy | how energized a song is in range 0 to 1 |
| key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1 |
| loudness | The overall loudness of a track in decibels (dB) |
| mode |  Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0 |
| speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks |
| acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic |
| instrumentalness | Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content |
| liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live |
| valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry) |
| tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration | 
| time_signature | An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4. | 

In [112]:
df_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162095 entries, 0 to 1162094
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1162095 non-null  object 
 1   followers   1162084 non-null  float64
 2   genres      1162095 non-null  object 
 3   name        1162092 non-null  object 
 4   popularity  1162095 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 44.3+ MB


&rarr; Some missing values in columns _followers_ and _name_!

In [113]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586672 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586672 non-null  int64  
 3   duration_ms       586672 non-null  int64  
 4   explicit          586672 non-null  int64  
 5   artists           586672 non-null  object 
 6   id_artists        586672 non-null  object 
 7   release_date      586672 non-null  object 
 8   danceability      586672 non-null  float64
 9   energy            586672 non-null  float64
 10  key               586672 non-null  int64  
 11  loudness          586672 non-null  float64
 12  mode              586672 non-null  int64  
 13  speechiness       586672 non-null  float64
 14  acousticness      586672 non-null  float64
 15  instrumentalness  586672 non-null  float64
 16  liveness          58

&rarr; Some missing values in column _name_!

In [114]:
df_artists.nunique()

id            1162095
followers       51998
genres          49155
name          1134429
popularity         99
dtype: int64

In [115]:
df_tracks.nunique()

id                  586672
name                446474
popularity             101
duration_ms         123122
explicit                 2
artists             114030
id_artists          115062
release_date         19700
danceability          1285
energy                2571
key                     12
loudness             29196
mode                     2
speechiness           1655
acousticness          5217
instrumentalness      5402
liveness              1782
valence               1805
tempo               122706
time_signature           5
dtype: int64

In [116]:
# get shape of the tracks dataframe

print(f"Track data: There are {df_tracks.shape[0]} observations and {df_tracks.shape[1]} feature variables ")
print('----------'*10)

df_tracks.shape

Track data: There are 586672 observations and 20 feature variables 
----------------------------------------------------------------------------------------------------


(586672, 20)

In [117]:
# check for number of null values in each columns

df_tracks.isnull().sum()

id                   0
name                71
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64

The tracks-csv has all the data we need. From this time on we will only work with the tracks data.
We will split the data to avoid data leakage.

In [118]:
# Drop NaNs in column name

df_tracks = df_tracks.dropna()

In [119]:
# re-check for number of null values in each columns

df_tracks.isnull().sum()

id                  0
name                0
popularity          0
duration_ms         0
explicit            0
artists             0
id_artists          0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

In [120]:
# check for duplicates

df_tracks.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
586667    False
586668    False
586669    False
586670    False
586671    False
Length: 586601, dtype: bool

In [121]:
df_tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


We have duplicated track names. Are these tracks with the same name from different artists, or du we have duplicates for the same track?

In [122]:
# Check for completely identical rows
identical_rows = df_tracks[df_tracks.duplicated(keep=False)]

# Print completely identical rows
print("Completely identical rows:")
print(identical_rows)

Completely identical rows:
Empty DataFrame
Columns: [id, name, popularity, duration_ms, explicit, artists, id_artists, release_date, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature]
Index: []


In [123]:
# get count of duplicated values in tracks dataframe

display(df_tracks.duplicated().value_counts())

False    586601
Name: count, dtype: int64

In [124]:
# show top 5 rows of data (transposed)

df_tracks.head().T

,0,1,2,3,4
id,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
name,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
popularity,6,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
explicit,0,0,0,0,0
artists,['Uli'],['Fernando Pessoa'],['Ignacio Corsini'],['Ignacio Corsini'],['Dick Haymes']
id_artists,['45tIt06XoI0Iio4LBEVpls'],['14jtPCOoNZwquk5wd9DxrY'],['5LiOoJbxVSAMkBS2fUm3X2'],['5LiOoJbxVSAMkBS2fUm3X2'],['3BiJGZsyX9sJchTqcSA7Su']
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


In [125]:
# Train-test Split

# Defining X and y
features = df_tracks.columns.tolist()
features.remove('name')

X = df_tracks[features]
y = df_tracks['name']

print(X.shape)
print(y.shape)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)  # , stratify=y)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

(586601, 19)
(586601,)
X_train: (439950, 19)
y_train: (439950,)
X_test: (146651, 19)
y_test: (146651,)


When we use 

 ```
stratify=y 
```

we get a ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2. 

This because of the nature of stratification. The stratify parameter set it to split data in a way to allocate test_size amount of data to each class. In this case, you don't have sufficient class labels of one of your classes to keep the data splitting ratio equal to test_size.

I confirm the above explanation. I have encountered this situation when dealing with a class that has a very low count . You can either take a random sample (not stratified) or try different test_size values, to be able to have an adequate size that could hold all your various labels.


---

# BASELINE MODELS SKETCH AND IMPLEMENTATION

------------------------------------

> We have 586672 observations and 20 feature variables in our current (initial) dataset

```python

all_features =  ['id', 'name',
    'popularity', 'duration_ms', 'explicit',
    'artists', 'id_artists','release_date',
    'danceability','energy', 'key',
    'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness',
    'liveness','valence','tempo',
    'time_signature']
```

#### Baseline model sketch


> baseline recommender: 

* baseline focus : 
    - similarity :  measure similarity based on valence mood category and genre, if popularity is greater than 0.7 then recommend top N song names and/or most similar content
    - top N : when grouped by valence mood category and genre, if popularity is greater than 0.7 then recommend top 10 song names and/or most similar content

* baseline features :
    - all features + genre? except id, id_artists
    - song name, acousticness, and popularity

* baseline features : all features + genre?

* recommend top 10 songs(name) based on similarity measure of genre (group by valence)


> next - main recommender model : 

* main model next :  mood groups/labels based on valence value
* recommend top N = 10 songs(name) based on mood label and popularity by genre/name/artist


* baseline recommender function:

    - similarity measure of track , genre , valence(category), artists
    - feat_col = ['valence', 'genre', 'name of music track', 'artists', 'popularity', acoust]
    - metric : similarity metric, top N
    - TSNE, KMeans, SVD, PCA

    - 


* Metric : Top_N, cosine similarity




In [126]:
# IMPORT LIBRARIES


try:

    import numpy as np
    import pandas as pd

    # databases - sql
    #from dotenv import dotenv_values
    #import sqlalchemy

    # visualisation
    import seaborn as sns
    import matplotlib.pyplot as plt

    # split data - avoid data leakage
    from sklearn.model_selection import train_test_split

    # preprocessing, scaling
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import MinMaxScaler

    # modelling
    from sklearn.cluster import KMeans

    # cross validation, hyperparameter tuning
    #from surprise.model_selection import GridSearchCV
    from sklearn.model_selection import GridSearchCV

    # metrics
    from sklearn.metrics import euclidean_distances
    from sklearn.metrics.pairwise import sigmoid_kernel
    from sklearn.metrics.pairwise import cosine_similarity

    # high dimensional usage - dimensionality reduction
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA

    # text converter/vectorizer
    from sentence_transformers import SentenceTransformer
    from sklearn.feature_extraction.text import CountVectorizer

    # pipeline
    from sklearn.pipeline import Pipeline



except ImportError as error:
    print(f"Installation of the required dependencies necessary! {error}")

    %pip install numpy
    %pip install pandas
    #%pip install dotenv
    #%pip install sqlalchemy
    %pip install seaborn
    %pip install matplotlib
    %pip install scikit-learn
    
    %pip install scikit-surprise
    %pip install sentence-transformers

    print(f"Successful installation of the required dependencies necessary")


    import warnings
    warnings.filterwarnings('ignore')



In [127]:
# all features

all_features =  ['id', 'name',
    'popularity', 'duration_ms', 'explicit',
    'artists', 'id_artists','release_date',
    'danceability','energy', 'key',
    'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness',
    'liveness','valence','tempo',
    'time_signature']



In [128]:
# descriptive statistics summary


df_tracks[all_features].describe().T.style \
        .format("{:.2f}")





,count,mean,std,min,25%,50%,75%,max
popularity,586601.00,27.57,18.37,0.00,13.00,27.00,41.00,100.00
duration_ms,586601.00,230054.85,126532.82,3344.00,175083.00,214907.00,263867.00,5621218.00
explicit,586601.00,0.04,0.21,0.00,0.00,0.00,0.00,1.00
danceability,586601.00,0.56,0.17,0.00,0.45,0.58,0.69,0.99
energy,586601.00,0.54,0.25,0.00,0.34,0.55,0.75,1.00
key,586601.00,5.22,3.52,0.00,2.00,5.00,8.00,11.00
loudness,586601.00,-10.21,5.09,-60.00,-12.89,-9.24,-6.48,5.38
mode,586601.00,0.66,0.47,0.00,0.00,1.00,1.00,1.00
speechiness,586601.00,0.10,0.18,0.00,0.03,0.04,0.08,0.97
acousticness,586601.00,0.45,0.35,0.00,0.10,0.42,0.78,1.00


In [129]:
# display data

df_tracks.head()


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4



### Clustering data based on mood (valence) and other audio features






In [130]:
#

# similarity recommend measure
# queries based on similar mood tracks
# modify column -- 
    # cols = valence .. n most high values of valence = 1. 
    # what are the top N mood-based-music in our data/ recommendation list?
# columns = ['mood_label', 'mood_class', 'music_name', 'acousticness','danceability','energy','instrumentalness','liveness','speechiness','valence']
# mood-emotions scale.. between 0 to 1 , group into 8 

# n_songs = 10 # 10 closes tracks neighbors
# filter_based_on_cols = ["valence", "acoustiness", "energy", "danceability"]
# sad_tracks = {"valence": 0, "acoustiness": 0}
# ... other moods and audio features
# happy_tracks = {"valence": 1, "acoustiness": 1 }

# search music track based on mood
# for music in mood_based_query( query_input = mood_based_data column).retrieve(top_n_songs):
    # trackname = get_track_name(moosic_data[name?])
    # search_music_name = search(music_name)

# mood based recommender
#music mood classification/prediction
# curate spotify music data to identifyand tag music with moods







In [131]:
# create new column 'mood labels'(text) and 'mood_class'(nominal value) 



#mood_labels = ["happy", "exuberant", "energetic", "frantic", "anxious/sad", "depression", "calm", "content"]
#mood_valence_values =  [1.0, 0.875, 0.75, 0.625, 0.5, 0.375, 0.25, 0.125, 0.0]

mood_labels = ["content", "calm", "depression", "anxious/sad", "frantic", "energetic", "exuberant", "happy"]
mood_class = [0, 1, 2, 3, 4, 5, 6, 7]
mood_valence_values =  [0.0 , 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]

# copy of data and info

moosic_data = df_tracks.copy(deep=True)

moosic_data["mood_labels"] = (pd.cut(moosic_data["valence"], bins=mood_valence_values, labels=mood_labels)).astype('string')
moosic_data["mood_class"] = (pd.cut(moosic_data["valence"], bins=mood_valence_values, labels=mood_class)).astype('Int64')
moosic_data = moosic_data.rename({'name': 'music_name', 'id': 'music_id', 'mode': 'modality', 'popularity': 'music_popularity'}, axis=1)
moosic_data.head(10)


,music_id,music_name,music_popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,modality,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood_labels,mood_class
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,...,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,calm,1
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,...,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,energetic,5
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,...,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,anxious/sad,3
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,...,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,anxious/sad,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,...,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,calm,1
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.227,0.2610,...,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4,content,0
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,['Francis Marty'],['2nuMRGzeJ5jJEKlfS7rZ0W'],1922,0.510,0.3550,...,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5,energetic,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,['Mistinguett'],['4AxgXfD7ISvJSTObqm4aIE'],1922,0.563,0.1840,...,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3,energetic,5
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,['Greg Fieler'],['5nWlsH5RDgFuRAiDeOFVmf'],1922,0.488,0.4750,...,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4,frantic,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-29,0.548,0.0391,...,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3,frantic,4


In [132]:
# specify features, drop id, id_artists
# unsupervised : kmeans clustering
# elbow method to get best number of clusters

df_moosic = moosic_data.copy(deep=True)
cat_data = df_moosic.select_dtypes(include=['object', 'string'])
cat_features = cat_data.columns.tolist()

print(cat_features)

num_data = df_moosic.select_dtypes(include=['int64', 'float64'])
num_features = num_data.columns.tolist()

print(num_features)




['music_id', 'music_name', 'artists', 'id_artists', 'release_date', 'mood_labels']
['music_popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'modality', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'mood_class']


In [133]:
# moosic baseline recommender : similarity based on feature vaiable 
#baseline focus : when grouped by valence mood category and genre, 
# if music is in similar queried mood_class as others then recommend top 10 song names and/or most similar content
# not normalize or scaled
# no outliers removed 

# work in progress!!! to-do


# test
''' 
baseline_params_0 = {
                'feat_cols' : ['music_name', 'popularity', 'danceability','energy',
                'loudness', 'acousticness', 'instrumentalness', 'tempo'],
                'target_col' : 'valence',
                'agg_type' : 'mean',
                'top_n' : 10
        }
'''

def baseline_moosic_v1(dataset, feat_cols= "mood_label", target_col = "music_name", agg_type = "count", top_n = 10, *args, **kwargs):

    data = dataset.groupby(feat_cols).agg({target_col : agg_type}).reset_index()
    data = data.sort_values(target_col, ascending=False)
    top_n_recommend = data.head(top_n)

    return top_n_recommend





In [134]:



# recommendations based on the valence (mood) feature
# mood_label, mood_class, name of music track


baseline_params_0 = {
                'feat_cols' : ['mood_labels', 'mood_class'],
                'target_col' : 'music_name',
                'agg_type' : 'count',
                'top_n' : 10
        }

baseline_0_stats = moosic_data[baseline_params_0['feat_cols']].describe().T.style \
        .format("{:.2f}")

display(baseline_0_stats)

print("_________"*10)

top_n_val = baseline_moosic_v1(moosic_data, **baseline_params_0)

top_n_val.sort_values(by=['mood_class'], ascending=True)



,count,mean,std,min,25%,50%,75%,max
mood_class,586232.00,3.91,2.07,0.00,2.00,4.00,6.00,7.00


__________________________________________________________________________________________


,mood_labels,mood_class,music_name
2,content,0,30604
1,calm,1,60156
3,depression,2,75501
0,anxious/sad,3,81006
6,frantic,4,89081
4,energetic,5,90271
5,exuberant,6,84796
7,happy,7,74817


In [135]:
# test
mood_filter = 'calm'
moosic_data['mood_labels'].str.contains(mood_filter).any()

True

In [138]:
## mood based baseline recommender
## recommend music based on similar mood classes/label

## Work in progress (WIP): 
    # mood filter or query can be a string based on mood label or mood class scale from 0 to 7
    # i.e. mood_filter = 'calm' or mood_filter = 1


def baseline_moosic_v2(dataset, mood_filter = 'calm', feat_cols= "mood_labels", 
                        target_col = "music_name", agg_type = "count", top_n = 10, *args, **kwargs):


    data = dataset.groupby(feat_cols).agg({target_col : agg_type}).reset_index()
    data = data.sort_values(target_col, ascending=False)

    if (isinstance(mood_filter, str)) and (data['mood_labels'].str.contains(mood_filter).any() == True):
        print(f'''Here are the top {top_n} music recommendations based on your current mood : {mood_filter} ''')

        for music in data.query(f" mood_labels == {mood_filter}").samples(top_n):
            top_n_recommend = music
            print(top_n_recommend)

    elif (isinstance(mood_filter, int)) and (data['mood_labels'].str.contains(mood_filter).any() == True):
        print(f'''Here are the top {top_n} music recommendations based on your current mood : {mood_filter} ''')

        for music in data.query(f" mood_labels == {mood_filter}").samples(top_n):
            top_n_recommend = music
            print(top_n_recommend)

    else:
        print('Error: enter mood eithwe by mood-label-text or mood-class-value')
    

    return top_n_recommend






In [ ]:

# recommendations for baseline model

mood_based_params_0 = {
                'feat_cols' : ['mood_labels', 'mood_class'],
                'target_col' : 'music_name',
                'agg_type' : 'count',
                'top_n' : 10
        }


moosic_baseline_stats = moosic_data[mood_based_params_0['feat_cols']].describe().T.style \
        .format("{:.2f}")

display(moosic_baseline_stats)

print("_________"*10)

moosic_baseline_recommender = baseline_moosic_v2(moosic_data, **mood_based_params_0)

moosic_baseline_recommender.sort_values(by=['mood_class'], ascending=True)




In [ ]:

#df_mdata = df_data.drop(columns=["id", "id_artists"], axis=1)
#df_mdata = df_data.drop(columns=[*cat_features], axis=1)

moosic_data_0 = moosic_data[['popularity', 'valence', 'tempo']]


plt.figure(figsize=(15,6))
sns.barplot(x=agex, y=agey, palette=['grey', 'grey','red','grey','grey'])
plt.title("Age (years)")
plt.xlabel("Age")
plt.ylabel("Number of Customer")
plt.show()

wcss_1 = []
for i in range(5, 20):
    km1 = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    km1.fit(df_mdata)
    wcss_1.append(km1.inertia_)
fig0=plt.figure(figsize=(10,6))  
fig0.patch.set_facecolor('#f6f5f5')

print('shape of data', df_mdata.shape)
#print('wcss1', wcss_1)

print('______'*20)

plt.plot(range(5, 20), wcss_1)
plt.title('The Elbow Method', fontsize = 20)
plt.xlabel('No. of Clusters')
plt.ylabel('wcss')
fig0.text(0.5,0.4,"The best k-value is here")
plt.show()




In [ ]:
# train and predict kmeans model with the best number of clusters values given, n_clusters = 6


km1 = KMeans(n_clusters = 6, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
y_means1 = km1.fit_predict(x1)

---

[algo] define  model based on similarity of possible target features and focus

---


In [ ]:
# TSNE (T-distributed Stochastic Neighbor Embedding) ML model 

"""
# T-SNE (T-distributed Stochastic Neighbor Embedding) ML model 

- unsupervised approach
- randomized nonlinear dimensionality reduction technique
- for high (multi-) dimensional data e.g N = 18 dimension space(asides id & artist id) for our data
- embeds high dimension data visually in a low dimension space of 2 or 3 dim
- i.e map high-dim data to low-dim space
- not affected by outliers

- algorithm :
    - randomized approach  for dimensionality reduction (not deterministic like PCA)
    - finds patterns in data based on similarity of data points with features 
    - similarity measure : pairwise conditional probability of data points choosing other data  with similar features as its neighbour
    - P(data point a | data point b) = P(data point a _n_ data point b) / P( data point b)
    - then minimize the sum of the difference between the similarities measured in high-dim and low-dim
    - represents datapoints properly in low-dim space

    - space/time complexity : quadratic time and space , inthe number of datapoints

- standardized data before usage to save time complexity needed for reduction process.



"""

#standardized_data = StandardScaler().fit_transform(data)
#print(standardized_data.shape)


#dftracks = df_tracks.drop(['id', 'id_artists'], axis = 1)
dftracks = df_data[['popularity', 'valence', 'tempo']]

model = TSNE(n_components = 2, random_state = 0)
tsne_data = model.fit_transform(dftracks.head(500))
plt.figure(figsize = (7, 7))
plt.scatter(tsne_data[:,0], tsne_data[:,1])
plt.show()




In [ ]:
dftracks0 = df_data[['popularity', 'valence', 'tempo']]

model0 = TSNE(n_components = 3, random_state = 0)
tsne_data0 = model0.fit_transform(dftracks0.head(500))
plt.figure(figsize = (7, 7))
plt.scatter(tsne_data0[:,0], tsne_data0[:,1])
plt.show()


In [ ]:
dftracks1 = df_data[['popularity', 'valence']] #df_data[['valence', 'tempo']]

model1 = TSNE(n_components = 2, random_state = 0)
tsne_data1 = model1.fit_transform(dftracks1.head(500))
plt.figure(figsize = (7, 7))
plt.scatter(tsne_data1[:,0], tsne_data1[:,1])
plt.show()

In [ ]:
standardized_data = StandardScaler().fit_transform(df_data[num_features])
print(standardized_data.shape)





In [ ]:
# Cosine similarity measure 

"""
# Cosine similarity measure 


- unsupervised approach
- randomized nonlinear dimensionality reduction technique
- for high (multi-) dimensional data e.g N = 18 dimension space(asides id & artist id) for our data
- embeds high dimension data visually in a low dimension space of 2 or 3 dim
- i.e map high-dim data to low-dim space
- not affected by outliers

- algorithm :
    - randomized approach  for dimensionality reduction (not deterministic like PCA)
    - finds patterns in data based on similarity of data points with features 
    - similarity measure : pairwise conditional probability of data points choosing other data  with similar features as its neighbour
    - P(data point a | data point b) = P(data point a _n_ data point b) / P( data point b)
    - then minimize the sum of the difference between the similarities measured in high-dim and low-dim
    - represents datapoints properly in low-dim space

    - space/time complexity : quadratic time and space , inthe number of datapoints




"""



